In [8]:
import sys
sys.path.append("..")

from src.train import train_model
from src.data import prep_data
from src.nets import Net

from torch.nn import CrossEntropyLoss, BCELoss
from torch.optim import Adam

from torch import nn
import torch.nn.functional as F

from torchvision import models


import torch
import math

In [ ]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.batch = nn.BatchNorm2d(2)
#         self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.fc1 = nn.Linear(64 * 18 * 18, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 2)
#         self.drp = nn.Dropout(0.4)

#     def forward(self, x):
#         x = self.batch(x)
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.drp(x)
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.drp(x)
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batch = nn.BatchNorm2d(2)
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(2048, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84, 1)
        self.drp = nn.Dropout(0.3)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.batch(x)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.drp(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.drp(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.drp(x)
        x = self.pool(F.relu(self.conv4(x)))
        x = self.drp(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sig(x)
        return x

In [11]:
#########
## VGG ##
#########

CFG = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name, n_classes, n_channels):
        super(VGG, self).__init__()
        self.features = self._make_layers(CFG[vgg_name], n_channels)
        self.n_classes = n_classes
        self.classifier = nn.Linear(2048, n_classes)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        if self.n_classes == 1:
            out = self.sig(out)
        return out

    def _make_layers(self, CFG, n_channels):
        layers = []
        in_channels = n_channels
        for x in CFG:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]

        return nn.Sequential(*layers)


In [14]:
net = VGG('VGG11', 1, 2)


epochs = 1 # for testing on laptop
criterion = BCELoss()

# adding weight_decay is a form of L2 regularization.
# See: https://discuss.pytorch.org/t/simple-l2-regularization/139
optimizer = Adam(net.parameters(), weight_decay=1e-5)#, lr=1e-3)

train_model(epochs, net, criterion, optimizer, early_stop=False)

  0%|                                                                                                                                                             | 0/1 [00:00<?, ?it/s]C:\Users\smithzd1\AppData\Local\Continuum\Anaconda3\envs\torch\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\smithzd1\AppData\Local\Continuum\Anaconda3\envs\torch\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[ loss: 0.6385 | acc: 0.6786 | vloss: 0.5929 | vacc: 0.6365 ]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:53<00:00, 173.15s/it]
C:\Users\smithzd1\AppData\Local\Continuum\Anaconda3\envs\torch\lib\site-packages\torch\serialization.py:158: UserWarning: Couldn't retrieve source code for container of type VGG. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

# for param in model_ft.parameters():
#     param.requires_grad = False
    
model_ft.fc = nn.Linear(num_ftrs, 2)

In [ ]:
model = models.vgg16(pretrained=True)